SUBMITTED BY: SHREYA PANDE

Installing tensorflow

In [2]:
pip install tensorflow tensorflow-datasets

Getting the version check ✔

In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
print(tf.__version__) #YOUR TF VERSION CHECK
print(tfds.__version__) #YOUR TFDS VERSION CHECK

2.12.0
4.9.2


1.1 Loading dataset and building input pipelines

In [4]:
def load_dataset():
    dataset_name = 'mnist'
    ds_train, ds_info = tfds.load(
        dataset_name,
        split='train[:90%]',
        shuffle_files=True,
        as_supervised=True,
        with_info=True
    )
    ds_test = tfds.load(
        dataset_name,
        split='train[90%:]',
        shuffle_files=True,
        as_supervised=True,
    )
    return ds_train, ds_test, ds_info

In [5]:
ds_train, ds_test, ds_info = load_dataset()
for image, label in ds_train.take(1):
    print(image.shape)

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.
(28, 28, 1)


1.2 Define CNN network model

In [24]:
from tensorflow.keras import layers, models

In [8]:
def create_cnn_model(input_shape, num_classes):
    model = tf.keras.Sequential([
        layers.Conv2D(32, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu', input_shape=input_shape),

        layers.Conv2D(64, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu'),
        layers.MaxPool2D(),

        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

def load_dataset():
    dataset_name = 'mnist'
    ds_train, ds_info = tfds.load(
        dataset_name,
        split='train[:90%]',
        shuffle_files=True,
        as_supervised=True,
        with_info=True
    )
    ds_test = tfds.load(
        dataset_name,
        split='train[90%:]',
        shuffle_files=True,
        as_supervised=True,
    )
    return ds_train, ds_test, ds_info

In [10]:
ds_train, ds_test, ds_info = load_dataset()
input_shape = (28, 28, 1)
num_classes = 10
model = create_cnn_model(input_shape, num_classes)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [11]:
ds_train = ds_train.map(lambda image, label: (tf.image.convert_image_dtype(image, tf.float32), label))
ds_train = ds_train.shuffle(10000)  # Shuffle the data
train_size = int(0.9 * ds_info.splits['train'].num_examples)  # 90% for training
val_size = ds_info.splits['train'].num_examples - train_size  # 10% for validation
ds_train = ds_train.take(train_size).batch(32)
ds_val = ds_train.skip(train_size).take(val_size).batch(32)

In [12]:
history = model.fit(ds_train, epochs=6, validation_data=ds_val)

Epoch 1/6
1688/1688 [==============================] - 229s 134ms/step - loss: 0.1168 - accuracy: 0.9632
Epoch 2/6
1688/1688 [==============================] - 257s 152ms/step - loss: 0.0379 - accuracy: 0.9884
Epoch 3/6
1688/1688 [==============================] - 219s 129ms/step - loss: 0.0237 - accuracy: 0.9920
Epoch 4/6
1688/1688 [==============================] - 212s 125ms/step - loss: 0.0144 - accuracy: 0.9955
Epoch 5/6
1688/1688 [==============================] - 215s 127ms/step - loss: 0.0113 - accuracy: 0.9963
Epoch 6/6
1688/1688 [==============================] - 218s 129ms/step - loss: 0.0090 - accuracy: 0.9969


In [13]:
ds_test = ds_test.map(lambda image, label: (tf.image.convert_image_dtype(image, tf.float32), label))
ds_test = ds_test.batch(32)
test_loss, test_acc = model.evaluate(ds_test)

188/188 [==============================] - 9s 41ms/step - loss: 0.0429 - accuracy: 0.9903


In [14]:
print("Validation accuracy:", history.history['accuracy'][-1])
print("Test accuracy:", test_acc)

Validation accuracy: 0.996925950050354
Test accuracy: 0.9903333187103271


QUESTION 2

In [18]:
def load_dataset():
    dataset_name = 'mnist'
    ds_train, ds_info = tfds.load(
        dataset_name,
        split='train[:90%]',
        shuffle_files=True,
        as_supervised=True,
        with_info=True
    )
    ds_test = tfds.load(
        dataset_name,
        split='train[90%:]',
        shuffle_files=True,
        as_supervised=True,
    )
    return ds_train, ds_test, ds_info


In [19]:
def normalize_image(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.0
    return image, label

def create_deep_mlp_model(input_shape, num_classes):
    model = models.Sequential([
        layers.Flatten(input_shape=input_shape),
        layers.Dense(300, activation='relu'),
        layers.Dense(100, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

In [22]:
def compile_and_train_model(model, train_ds, val_ds):
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    EPOCHS = 6
    history = model.fit(train_ds, epochs=EPOCHS, validation_data=val_ds)
    return history

def evaluate_model(model, test_ds):
    test_loss, test_accuracy = model.evaluate(test_ds)
    return test_accuracy


In [25]:
ds_train, ds_test, ds_info = load_dataset()
input_shape = (28, 28, 1)
num_classes = 10
ds_train = ds_train.map(normalize_image)
ds_train = ds_train.shuffle(10000)
train_size = int(0.9 * ds_info.splits['train'].num_examples)
val_size = ds_info.splits['train'].num_examples - train_size
ds_train = ds_train.take(train_size).batch(32)
ds_val = ds_train.skip(train_size).take(val_size).batch(32)

ds_test = ds_test.map(normalize_image)
ds_test = ds_test.batch(32)
deep_mlp_model = create_deep_mlp_model(input_shape, num_classes)

In [26]:
history = compile_and_train_model(deep_mlp_model, ds_train, ds_val)

Epoch 1/6
1688/1688 [==============================] - 23s 12ms/step - loss: 0.2120 - accuracy: 0.9366
Epoch 2/6
1688/1688 [==============================] - 15s 8ms/step - loss: 0.0857 - accuracy: 0.9729
Epoch 3/6
1688/1688 [==============================] - 14s 8ms/step - loss: 0.0579 - accuracy: 0.9810
Epoch 4/6
1688/1688 [==============================] - 13s 8ms/step - loss: 0.0416 - accuracy: 0.9866
Epoch 5/6
1688/1688 [==============================] - 14s 8ms/step - loss: 0.0335 - accuracy: 0.9892
Epoch 6/6
1688/1688 [==============================] - 14s 8ms/step - loss: 0.0271 - accuracy: 0.9912


In [27]:
test_accuracy = evaluate_model(deep_mlp_model, ds_test)
test_accuracy

188/188 [==============================] - 1s 5ms/step - loss: 0.1003 - accuracy: 0.9763


0.9763333201408386

QUESTION 3

In [28]:
def load_dataset():
    dataset_name = 'mnist'
    ds_train, ds_info = tfds.load(
        dataset_name,
        split='train[:90%]',
        shuffle_files=True,
        as_supervised=True,
        with_info=True
    )
    ds_test = tfds.load(
        dataset_name,
        split='train[90%:]',
        shuffle_files=True,
        as_supervised=True,
    )
    return ds_train, ds_test, ds_info

def normalize_image(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.0
    return image, label

def create_mlp_model(input_shape, num_classes):
    model = models.Sequential([
        layers.Flatten(input_shape=input_shape),
        layers.Dense(100, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model


In [29]:
def compile_and_train_model(model, train_ds, val_ds):
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    EPOCHS = 6
    history = model.fit(train_ds, epochs=EPOCHS, validation_data=val_ds)
    return history
def evaluate_model(model, test_ds):
    test_loss, test_accuracy = model.evaluate(test_ds)
    return test_accuracy
ds_train, ds_test, ds_info = load_dataset()
input_shape = (28, 28, 1)
num_classes = 10
ds_train = ds_train.map(normalize_image)
ds_train = ds_train.shuffle(10000)
train_size = int(0.9 * ds_info.splits['train'].num_examples)
val_size = ds_info.splits['train'].num_examples - train_size
ds_train = ds_train.take(train_size).batch(32)
ds_val = ds_train.skip(train_size).take(val_size).batch(32)


In [30]:
ds_test = ds_test.map(normalize_image)
ds_test = ds_test.batch(32)
mlp_model = create_mlp_model(input_shape, num_classes)
history = compile_and_train_model(mlp_model, ds_train, ds_val)
test_accuracy = evaluate_model(mlp_model, ds_test)

Epoch 1/6
1688/1688 [==============================] - 11s 6ms/step - loss: 0.2899 - accuracy: 0.9170
Epoch 2/6
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1320 - accuracy: 0.9601
Epoch 3/6
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0915 - accuracy: 0.9725
Epoch 4/6
1688/1688 [==============================] - 8s 4ms/step - loss: 0.0704 - accuracy: 0.9793
Epoch 5/6
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0546 - accuracy: 0.9839
Epoch 6/6
188/188 [==============================] - 1s 4ms/step - loss: 0.0877 - accuracy: 0.9757


In [31]:
test_accuracy

0.9756666421890259